Fermi SM关键组件：
CUDA核心
共享内存/一级缓存
寄存器文件
加载/存储单元
特殊功能单元
线程束调度器

GPU->多个SM->数百个线程并发运行（线程块调度）->同一线程指令级并行

CUDA->SIMT架构管理执行线程->32个线程为一组->线程束(warp)
SIMT区别于SIMD：
线程有自己的指令PC
线程有自己的寄存器状态
线程有独立的执行路径

线程块被调度到SM上直到执行完成，一个SM同时容纳多个线程块，可能不同速度前进。
SM中并发的线程束之间切换没有开销，因为硬件资源已经被分配到线程块中。

fermi架构：
GPU计算

线程块：
<p>逻辑角度：线程的集合，可组织为一维二维三维</p>
<p>硬件角度：一维线程束的集合。线程块组织为一维布局，32个连续线程为一个线程束。</p>

<h4>3.2.2线程束分化</h4>
<p>GPU无CPU式分支预测机制，一个线程束中所有线程必须执行相同的命令</p>
<p>同一线程束线程执行不同的命令为线程束分化，造成性能年下降</p>


<h4>3.2.3资源分配</h4>
<p>线程束本地执行上下文资源：程序计数器 寄存器 共享内存 SM中线程束切换上下文无损失</p>
<p>SM寄存器大小 SM共享内存为固定值 线程消耗寄存器越多 线程束越少 同消耗内存</p>

<p>线程束类型:活跃的线程束 阻塞的线程束 符合条件的线程束</p>
<p>同时满足32个CUDA核心可用于执行 当前指令所有参数准备就绪 活跃的线程束小于某值 选定的线程束小于某值</p>



<h4>3.2.4延迟隐藏</h4>
<p>指令发出和完成之间的始终周期->指令延迟</p>
<p>每个时钟周期所有线程调度器都有一个符合条件的线程束 可达到计算资源的完全利用->其他常住线程束发布指令可隐藏每个指令的延迟</p>
<p>指令分为：算术指令 内存指令</p>
<p>算术操作->10-20周期 全局内存访问->400-800周期</p>
<p>指令级并行（ILP）/线程级并行（TLP）</p>

<h4>3.2.5占用率</h4>
<p>活跃线程束数量/最大线程束数量</p>
<p>网格和线程块大小准则</p>
<p>每个块中线程数量是线程束大小(32)的倍数</p>
<p>每个块至少128或256线程</p>
<p>根据内核资源需求调整块大小</p>
<p>块数量远多于SM数量->足够并行</p>

<h4>3.2.6同步</h4>
<p>两个级别同步：</p>
系统级：等待主机和设备完成所有的工作 cudaDeviceSynchronize(void)<br>
块级：设备执行过程中等待一个线程块中所有线程到达同一点<br>
线程块中线程可通过共享内存和寄存器来共享数据<br>
同一线程块间线程同步<br>
不同块之间用全局同步<br>

<h4>3.2.7可扩展性</h4>
<p>网格中线程块以并行或连续或任意的顺序被执行->CUDA程序在任意数量计算核心间可以扩展</p>

<h3>3.3并行性的表现</h3>

nvprof --metrics achieved_occupancy<br>
nvprof --metrics gld_throughput<br>
nvprof --metrics gld__efficiency<br>
增大并行性<br>
线程块个数增加，线程块最内层维度变化，

<h3>3.4避免分支分化</h3>
<h4>3.4.1并行归约问题</h4>
<h5>相邻配对</h5>
并行归约中会出现线程分化，即一个线程块中执行加法的有不执行加法的
改善方法为在每一轮并行归约中使用不同的线程块中的线程束，如第一轮使用一个线程块中的前8个线程束，第二轮用前4个等等，直到须使用的线程数不到一个线程束时才会出现线程分化。
<h5>3.4.4交错配对的归约</h5>
初始跨度每次减半，比相邻配对快

<h3>3.5展开循环</h3>
<p>展开线程->即一个线程控制两个数据块 此时块索引数字除以2</p>
<p>完全展开->即将循环全部展开，使其在一次全部完成</p>
<p>模板函数归约->即将用模板函数展开循环 可以进一步减少分支消耗</p>

<h3>3.6动态并行</h3>
<p>父网格/线程块/线程配置和生成子网格/线程块/线程</p>
<p>线程创建的 所有子网格完成后，父网格才会完成 在父网格中设置栅栏可使其与子网格显式同步 父网格和子网格共享相同的全局和常量内存存储 但有不同的局部内存和共享内存</p>